In [1]:
import oatpy as oat
import plotly.graph_objects as go
import numpy as np

In [2]:
#   SET PARAMETER VALUES
npoints_per_circle  =   80
maxdis              =   None
maxdim              =   1

#   GENERATE CLOUD FROM RANDOM SEED
cloud               =   []
for seed in range(3):
    circle          =   oat.point_cloud.annulus(npoints=npoints_per_circle, rad0=1, rad1=2.5, random_seed=seed)
    circle[:,0]     +=  2 * np.cos( seed * 2 * np.pi / 4)
    circle[:,1]     +=  2 * np.sin( seed * 2 * np.pi / 4)    
    cloud.append( circle )
cloud               =   np.concatenate(cloud)

#   PLOT
trace               =   go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers")
fig                 =   go.Figure([trace])
fig.update_layout( title=dict(text="Three circles, with noise"), width=800, height=800 )
fig.show()

# Compute persistent homology

We compute persistent homology by factoring the boundary matrix.  The following cell generates a sparse distance matrix and feeds it to the persistent homology solver.  The result is a factored boundary matrix.  We will extract information from this matrix in the following cells.

In [3]:
# the minimum enclosing radius; all homology vanishes above this filtration parameter
enclosing               =   oat.dissimilarity.enclosing_from_cloud(cloud)   

# distance matrix with values over enclosing + 0.0000000001 removed; adding 0.0000000001 avoids problems due to numerical error
dissimilairty_matrix    =   oat.dissimilarity.matrix_from_cloud(            
                                cloud                       =   cloud,
                                dissimilarity_max           =   enclosing + 0.0000000001,
                            )

# build and factor the boundary matrix
factored                =   oat.rust.FactoredBoundaryMatrixVr( 
                                dissimilarity_matrix        =   dissimilairty_matrix,
                                homology_dimension_max      =   1,
                            )

Full details on `FactoredBoundaryMatrixVr` can be retreived with Python's `help` function.

In [4]:
help(oat.rust.FactoredBoundaryMatrixVr)

Help on class FactoredBoundaryMatrixVr in module builtins:

class FactoredBoundaryMatrixVr(object)
 |  Methods defined here:
 |  
 |  boundary_matrix(self, /)
 |      Returns the boundary matrix, formatted as a `scipy.sparse.csr_matrix` with rows/columns
 |      labeled by simplices.
 |      
 |      The indices for this matrix can be retrieved with `self.indices_boundary_matrix()`
 |  
 |  comb_domain(self, /)
 |      Returns the domain COMB, formatted as a `scipy.sparse.csr_matrix` with rows/columns
 |      labeled by simplices.
 |      
 |      The indices for this matrix can be retrieved with `self.indices_boundary_matrix()`
 |  
 |  homology(self, /, return_cycle_representatives, return_bounding_chains)
 |      Extract a barcode and a basis of cycle representatives
 |      
 |      Computes the persistent homology of the filtered clique complex (ie VR complex)
 |      with dissimilarity matrix `dissimilarity_matrix`, over the field of rational numbers.  
 |      
 |      - Edges o

In [5]:
# the minimum enclosing radius; all homology vanishes above this filtration parameter
enclosing               =   oat.dissimilarity.enclosing_from_cloud(cloud)   

# distance matrix with values over enclosing + 0.0000000001 removed; adding 0.0000000001 avoids problems due to numerical error
dissimilairty_matrix    =   oat.dissimilarity.matrix_from_cloud(            
                                cloud               =   cloud,
                                dissimilarity_max   =   enclosing + 0.0000000001,
                            )

# build and factor the boundary matrix
factored                =   oat.rust.FactoredBoundaryMatrixVr( 
                                dissimilarity_matrix        =   dissimilairty_matrix,
                                homology_dimension_max      =   1,
                            )

# Plot the barcode

In [6]:
homology    =   factored.homology(
                    return_cycle_representatives    =   True,
                    return_bounding_chains          =   True,
                )
fig         =   oat.plot.pd( homology )
fig.show()

For convenience, export information about the barcode to a data frame.

# Inspect a cycle representatives and bounding chains

The `homology` object is a Pandas data frame containing information about cycles.

In [7]:
display(homology)

dimension     birth     death birth simplex    death simplex  \
id                                                                  
0            0  0.000000  0.244575         [239]       [201, 239]   
1            0  0.000000  0.120144         [238]       [119, 238]   
2            0  0.000000  0.146891         [237]        [62, 237]   
3            0  0.000000  0.090698         [236]       [203, 236]   
4            0  0.000000  0.136732         [235]        [79, 235]   
..         ...       ...       ...           ...              ...   
278          1  0.365403  0.503499     [52, 129]   [52, 100, 129]   
279          1  0.304325  0.322749    [173, 221]  [173, 188, 221]   
280          1  0.258583  0.272611     [16, 119]  [119, 229, 238]   
281          1  0.255000  0.417789    [100, 129]    [24, 47, 100]   
282          1  0.119940  0.159975      [38, 47]    [87, 99, 105]   

                                  cycle representative  cycle nnz  \
id                                                                  
0      simplex  filtration coefficient
0   [239]   ...          2   
1      simplex  filtration coefficient
0   [238]   ...          2   
2      simplex  filtration coefficient
0   [237]   ...          2   
3      simplex  filtration coefficient
0   [236]   ...          2   
4      simplex  filtration coefficient
0   [235]   ...          2   
..                                                 ...        ...   
278         simplex  filtration coefficient
0    [5...         15   
279        simplex  filtration coefficient
0  [173,...          4   
280        simplex  filtration coefficient
0   [16,...          4   
281        simplex  filtration coefficient
0  [100,...         10   
282       simplex  filtration coefficient
0   [38, ...          5   

                                        bounding chain  bounding nnz  
id                                                                    
0          simplex  filtration coefficient
0  [201,...           1.0  
1          simplex  filtration coefficient
0  [119,...           1.0  
2         simplex  filtration coefficient
0  [62, 2...           2.0  
3          simplex  filtration coefficient
0  [203,...           1.0  
4         simplex  filtration coefficient
0  [79, 2...           1.0  
..                                                 ...           ...  
278             simplex  filtration coefficient
0  ...          15.0  
279             simplex  filtration coefficient
0  ...           2.0  
280             simplex  filtration coefficient
0  ...           2.0  
281            simplex  filtration coefficient
0   ...           8.0  
282           simplex  filtration coefficient
0  [8...           3.0  

[283 rows x 9 columns]

In [8]:
homology["cycle representative"][280]

simplex  filtration coefficient
0   [16, 119]    0.258583          -1
1   [16, 229]    0.223198           1
2  [229, 238]    0.212329           1
3  [119, 238]    0.120144          -1

In [9]:
homology["bounding chain"][280]

simplex  filtration coefficient
0  [119, 229, 238]    0.272611           1
1   [16, 119, 229]    0.272611          -1

# Optimize a cycle

Let's optimize the cycle that has birth simplex [9,19]

In [10]:
# optimize the cycle
optimal     =   factored.optimize_cycle(
                    birth_simplex                   =   homology["birth simplex"][240], 
                    problem_type                    =   "preserve PH basis",
                )

# display the data frame that contains the solution
display(optimal)

# display the data frame for just the optimal cycle (which is contained in the larger data frame)
display(optimal["chain"]["optimal cycle"])


Finished construcing L1 optimization program.
Constraint matrix has 10369 nonzero entries.
Passing program to solver.

Done solving.
MINILP solution: Solution { direction: Minimize, num_vars: 3418, num_constraints: 3658, objective: 9.144927301597422 }


cost  nnz  \
type of chain                                    
initial cycle                   17.472302   59   
optimal cycle                    9.144927   12   
difference in bounding chains         NaN   85   
difference in essential chains        NaN    0   
Ax + z - y                            NaN    0   

                                                                            chain  
type of chain                                                                      
initial cycle                          simplex  filtration coefficient
0   [20...  
optimal cycle                          simplex  filtration coefficient
0   [20...  
difference in bounding chains               simplex  filtration coefficient
0 ...  
difference in essential chains  Empty DataFrame
Columns: [simplex, filtration,...  
Ax + z - y                      Empty DataFrame
Columns: [simplex, filtration,...

simplex  filtration coefficient
0   [201, 206]    0.957614           1
1   [189, 228]    0.948681          -1
2   [197, 206]    0.940493          -1
3   [201, 217]    0.877634          -1
4   [183, 184]    0.861983           1
5   [113, 209]    0.842895           1
6   [156, 186]    0.822437           1
7   [156, 209]    0.684230          -1
8   [183, 186]    0.631053          -1
9   [184, 217]    0.623784           1
10  [197, 228]    0.524042           1
11  [113, 189]    0.430080          -1

# Plot an optimal cycle

<span style="color:red">Try toggling the legend entries to make different parts of the calculation appear / disappear!</span>.

In [11]:
optimal             =   factored.optimize_cycle(
                            birth_simplex                   =   homology["birth simplex"][240], 
                            problem_type                    =   "preserve PH basis",
                        )


edges_initial       =   optimal["chain"]["initial cycle"]["simplex"].tolist();
edges_optimal       =   optimal["chain"]["optimal cycle"]["simplex"].tolist();
triangles           =   optimal["chain"]["difference in bounding chains"]["simplex"].tolist()
coo                 =   cloud # coo stands for coordinate oracl e

traces_initial      =   [ oat.plot.edge__trace2d( edge, coo  ) for edge in edges_initial ];
traces_optimal      =   [ oat.plot.edge__trace2d( edge, coo  ) for edge in edges_optimal ];
traces_differe      =   [ oat.plot.triangle__trace2d( triangle=tri, coo=coo ) for tri in triangles ];
trace_cloud         =   go.Scatter(x=cloud[:,0], y=cloud[:,1], mode="markers") # the point cloud

trace_cloud.update(showlegend=True, opacity=0.5, name="Point cloud")
for n, trace in enumerate(traces_initial): trace.update(showlegend=(n==0), legendgroup="init", opacity=0.5, name="Initial cycle", line=dict(color="black",dash="dash"))
for n, trace in enumerate(traces_optimal): trace.update(showlegend=(n==0), legendgroup="opti", opacity=0.5, name="Optimal cycle", line=dict(color="black"))
for n, trace in enumerate(traces_differe): trace.update(showlegend=(n==0), legendgroup="diff", opacity=0.2, name="Bounding difference",)

fig = go.Figure(data= [trace_cloud] + traces_initial + traces_optimal + traces_differe )
fig.update_layout(title=dict(text="Optimization of a cycle representative"))
fig.update_layout(height=800,width=850) 
fig.show()


Finished construcing L1 optimization program.
Constraint matrix has 10369 nonzero entries.
Passing program to solver.

Done solving.
MINILP solution: Solution { direction: Minimize, num_vars: 3418, num_constraints: 3658, objective: 9.144927301597425 }


# Caution: "disappearing" triangles

OAT offers a function to plot multiple triangle in a sincle trace, however Plotly's opacity feature will sometimes cause overlapping triangles to "disappear."

You can see this in the following example.

In [12]:
optimal             =   factored.optimize_cycle( 
                            birth_simplex   =   homology["birth simplex"][246], 
                            problem_type    =   "preserve PH basis"
                        )

edges_initial = optimal["chain"]["initial cycle"]["simplex"].tolist();
edges_optimal = optimal["chain"]["optimal cycle"]["simplex"].tolist();
triangles = optimal["chain"]["difference in bounding chains"]["simplex"].tolist()
coo = cloud # coo stands for coordinate oracle

traces_initial      = [ oat.plot.edge__trace2d( edge, coo  ) for edge in edges_initial ];
traces_optimal      = [ oat.plot.edge__trace2d( edge, coo  ) for edge in edges_optimal ];
trace_difference    = oat.plot.triangles__trace2d( triangles, coo ) # the chain that bounds the difference between the cycles
trace_cloud         = go.Scatter(x=cloud[:,0], y=cloud[:,1], mode="markers") # the point cloud

trace_cloud.update(showlegend=True, opacity=0.5, name="Point cloud")
for n, trace in enumerate(traces_initial): trace.update(showlegend=(n==0), legendgroup="init", opacity=0.5, name="Initial cycle", line=dict(color="black",dash="dash"))
for n, trace in enumerate(traces_optimal): trace.update(showlegend=(n==0), legendgroup="opti", opacity=0.5, name="Optimal cycle", line=dict(color="black"))
trace_difference.update(showlegend=True, opacity=0.1, name="Bounding difference", fillcolor="black", line=dict(color="black"))

fig = go.Figure(data= [trace_cloud] + traces_initial + traces_optimal + [trace_difference] )
fig.update_layout(title=dict(text="Optimization of a cycle representative"))
fig.update_layout(height=800,width=850) 
fig.show()


Finished construcing L1 optimization program.
Constraint matrix has 5124 nonzero entries.
Passing program to solver.

Done solving.
MINILP solution: Solution { direction: Minimize, num_vars: 2170, num_constraints: 2408, objective: 9.055735922031275 }


# Fix opacity, and add hover text to triangles

If you'd like all the triangles to have the same color, instead of placing them in the same trace you can assign each the same color in a trace update:

In [13]:
optimal             =   factored.optimize_cycle( homology["birth simplex"][240], problem_type="preserve PH basis")

edges_initial = optimal["chain"]["initial cycle"]["simplex"].tolist();
edges_optimal = optimal["chain"]["optimal cycle"]["simplex"].tolist();
triangles = optimal["chain"]["difference in bounding chains"]["simplex"].tolist()
coo = cloud # coo stands for coordinate oracle

traces_initial      = [ oat.plot.edge__trace2d( edge, coo  ) for edge in edges_initial ];
traces_optimal      = [ oat.plot.edge__trace2d( edge, coo  ) for edge in edges_optimal ];
traces_differe      = [ oat.plot.triangle__trace2d( triangle=tri, coo=coo ) for tri in triangles ];
trace_cloud         = go.Scatter(x=cloud[:,0], y=cloud[:,1], mode="markers") # the point cloud

trace_cloud.update(showlegend=True, opacity=0.5, name="Point cloud")
for n, trace in enumerate(traces_initial): trace.update(showlegend=(n==0), legendgroup="init", opacity=0.5, name="Initial cycle", line=dict(color="white",dash="dash"))
for n, trace in enumerate(traces_optimal): trace.update(showlegend=(n==0), legendgroup="opti", opacity=0.5, name="Optimal cycle", line=dict(color="white"))
for n, trace in enumerate(traces_differe): trace.update(showlegend=(n==0), legendgroup="diff", opacity=0.5, name="Bounding difference", hovertext="Simplex "+str(triangles[n]), fillcolor="white" )

fig = go.Figure(data= [trace_cloud] + traces_initial + traces_optimal + traces_differe )
fig.update_layout(title=dict(text="Optimization of a cycle representative"))
# fig.update_layout(height=800,width=850) 

fig.update_layout(
        title=dict(text="Cycle representative and bounding chain"),
        template="plotly_dark",
        height=1200,
        width=1600,
    )

fig.show()


Finished construcing L1 optimization program.
Constraint matrix has 10369 nonzero entries.
Passing program to solver.

Done solving.
MINILP solution: Solution { direction: Minimize, num_vars: 3418, num_constraints: 3658, objective: 9.144927301597424 }
